In [1]:
import pandas as pd
import numpy as np

In [11]:
df = pd.read_csv('clean_data.csv')
del df['Unnamed: 0']
del df['opening_pgn']
del df['pgn']
del df['game_id']
df

,eco,opening,white_rating,black_rating,white_result,black_result,time_class,time_control,white_bracket,black_bracket
0,A21,English,1330,1384,checkmated,win,blitz,180,1300,1300
1,B01,Scandinavian,921,833,win,checkmated,blitz,300,900,800
2,A00,Uncommon Opening,1003,444,win,checkmated,bullet,60,1000,400
3,C23,Bishop's Opening,496,351,win,checkmated,blitz,180,400,300
4,A45,Queen's Pawn Game,1814,1761,win,resigned,rapid,600,1800,1700
...,...,...,...,...,...,...,...,...,...,...
9779,C20,King's Pawn Game,648,675,win,checkmated,rapid,600,600,600
9780,A10,English,1226,1197,win,timeout,bullet,60,1200,1100
9781,B10,Caro-Kann,887,864,resigned,win,rapid,600,800,800
9782,A01,Nimzovich-Larsen Attack,1799,1731,win,resigned,blitz,180,1700,1700


In [12]:
print("hello world")

hello world
